In [2]:
# Leer el archivo .env con la clave API
from dotenv import load_dotenv
load_dotenv()

True

#### Instrucciones de formato
Propósito del ejemplo   <br>
Aprender a recuperar las instrucciones de formato incluidas por defecto en los diversos analizadores sintácticos.


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import DatetimeOutputParser
from langchain.output_parsers import XMLOutputParser

print("JSON - ", JsonOutputParser().get_format_instructions())
print("***************************************************")
print("CSV - ", CommaSeparatedListOutputParser().get_format_instructions())
print("***************************************************")
print("Fecha - ", DatetimeOutputParser().get_format_instructions())
print("***************************************************")
print("XML - ", XMLOutputParser().get_format_instructions())
print("***************************************************")
print(StrOutputParser().get_format_instructions())

##### Analizador StrOutputParser
Propósito del ejemplo  <br>
Mostrar el uso del analizador de uso más frecuente y el más sencillo.


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

mixtral = ChatGroq(model="mixtral-8x7b-32768",
                   temperature=1,
                   max_tokens=250)

prompt = ChatPromptTemplate.from_template(
    "Eres un pirata extrovertido y entusiasta que ha viajada el mundo en busqueda \
    de tesoros culinarios y aventuras. Cuentame acerca de una de tus aventuras como \
    pirata en Argentina acerca de {tema}")

output_parser = StrOutputParser()

# Se reunen los tres componentes con una cadena single chain using LCEL
chain = prompt | mixtral | output_parser
chain = chain.invoke({"tema": "empanadas"})
chain

##### Desglose de las salidas

In [ ]:
prompt_value = prompt.invoke({"tema": "empanadas"})
print("Ver tipo de dato de salida: ", type(prompt_value))
print("*****************************")
# PromptValue es versatil
print("Convertir a mensajes: ", prompt_value.to_messages())
print("*****************************")
print("Convertir a cadena de texto: ", prompt_value.to_string())


In [ ]:
mensaje = mixtral.invoke(prompt_value)
print("Tipo de dato de salida de invoke con el modelo: ", type(mensaje))

In [ ]:
salida_parser = output_parser.invoke(mensaje)
print(type(salida_parser))

##### Analizador CommaSeparatedListOutputParser
Propósito del ejemplo   <br>
Mostrar la aplicación del analizador sintáctico segundo en frecuencia de uso.

In [3]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

mixtral = ChatGroq(model="mixtral-8x7b-32768", temperature=0)
parser_csv = CommaSeparatedListOutputParser()

In [4]:
# Recuperar las instrucciones de formato
format_instructions = parser_csv.get_format_instructions()
# Se crea la plantilla de prompt
prompt_template = PromptTemplate(
    template="Genere un listado en español con tres objetos {objetos}.\n{format_instructions}",
    input_variables=["objetos"],
    partial_variables={"format_instructions": format_instructions})

In [6]:
chain = prompt_template | mixtral | parser_csv    #| parser_csv
salida = chain.invoke({"objetos":"Objetos de geometria prismática."})
print(salida)

['Prisma rectangular', 'prisma pentagonal', 'prisma hexagonal']


##### Analizador CommaSeparatedListOutputParser Ejemplo 2
Construir una tabla

In [24]:
import csv
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
import pandas as pd

# Inicializar el analizador parser y el LLM
output_parser = CommaSeparatedListOutputParser()
mixtral = ChatGroq(model="mixtral-8x7b-32768", temperature=0)

# Crear la plantilla de prompt
prompt = PromptTemplate(
    template="Haga un listado de 5 libros populares usando el formato 'Titulo por Autor'.\n{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Generar la lista
response = llm(prompt.format())
book_list = output_parser.parse(response)

# Crear el archivo CSV 
csv_filename = "directorio/libros_populares.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Titulo", "Autor"])  # Header
    for libro in book_list:
        partes = libro.split(" por ", 1)  # Split en el primer " por "
        if len(partes) == 2:
            titulo, autor = partes
            writer.writerow([titulo.strip(), autor.strip()])
        else:
            print(f"Hubo un problema.")

# Leer el archivo CSV a un  DataFrame de pandas
df = pd.read_csv(csv_filename)

# Mostrar la tabla
print("\nTabla de libros:")
print(df.to_string(index=False))


Tabla de libros:
            Titulo                                                                                                                                                                                                                               Autor
1. "El principito" Antoine de Saint-Exupéry\n2. "Cien años de soledad" por Gabriel García Márquez\n3. "Harry Potter y la piedra filosofal" por J.K. Rowling\n4. "El señor de los anillos" por J.R.R. Tolkien\n5. "Orgullo y prejuicio" por Jane Austen


#### Partial_variables versus input_variables
Hemos visto que el parámetro input_variables de PromptTemplate se emplea para declarar variables dinámicas. Indica cuales son y cómo se llaman. Se ejecuta cada vez que se invoque la plantilla. Si no hay variables dinámicas se le asigna una lista vacía. Es obligatorio a menos que se use from_template.   <br>
El parámetro partial_variables es opcional. Suele incorporarse cuando los valores de determinadas variables dinámicas se conocen de antemano o pueden ser creados dinámicamente. Así, el programa puede cargar los valores conocidos, ganando tiempo de ejecución.  <br>
partial_variables es un diccionario donde la clave es el nombre de la variable y el valor es un valor estático o una función que devuelva un valor. 
Usar input_variables cuando los valores cambian con el prompt; usar partial_variables si el valor es persistente, e.g., clave API, o puede ser generado automáticamente.   <br>
No obstante, es legal incluir "format_instructions" en input_variables junto con “objetos”. 

In [30]:
# Insercion automatica de la fecha
from langchain.prompts import PromptTemplate
import datetime, locale

locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')
fecha_actual = lambda: datetime.date.today().strftime("%B %d, %Y")

template = PromptTemplate(
    input_variables=["topic"],
    partial_variables={"date": fecha_actual },
    template="Hoy es {date}. Escriba sobre {topic}."
)

prompt = template.format(topic="Peru")
print(prompt)

Hoy es septiembre 23, 2024. Escriba sobre Peru.


##### Analizador DatetimeOutputParser
Propósito del ejemplo   <br>
Mostrar la aplicación del analizador sintáctico especializado en procesar fechas y tiempo.

In [43]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")
parser_fecha = DatetimeOutputParser()
template = """Responda la pregunta:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template=template,
    partial_variables={"format_instructions": parser_fecha.get_format_instructions()},
)

chain = prompt | llm | parser_fecha

res = chain.invoke({"question": "Cuando se crearon las Naciones Unidas?"})
print(res)

1945-10-24 12:01:00


In [38]:
res = prompt | llm
res

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1236-12-19T17:42:24.055565Z, 1107-01-04T22:37:17.748360Z, 0552-05-05T03:37:40.549675Z\n\nReturn ONLY this string, no other words!"}, template='Responda la pregunta:\n\n{question}\n\n{format_instructions}')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FF5FABBED0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FF5FB040D0>, temperature=1e-08, groq_api_key=SecretStr('**********'))

In [40]:
res2 = prompt | llm
res2

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0174-10-20T19:32:23.868052Z, 0669-12-09T19:14:30.971687Z, 1578-04-01T14:49:13.576485Z\n\nReturn ONLY this string, no other words!"}, template='Responda la pregunta:\n\n{question}\n\n{format_instructions}')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FF5FAFD9D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FF5FAFE450>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [42]:
raw_response = llm(prompt.format(question="¿Cuándo se crearon las Naciones Unidas?"))
# print("Raw response:", raw_response)
# res = parse_date(raw_response)
# print("Parsed date:", res)

TypeError: Got unknown type R